In [1]:
import xarray as xr
import numpy as np
import glob
import os
import dask

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import cartopy
cartopy.config['pre_existing_data_dir']  = '/home/ad/alberta/git/cartopy_data' 


In [2]:
phase='science_phase' # calval_phase or science_phase
simu='eNATL60-BLB002' # or eNATL60-BLBT02
data='karin' # or nadir

In [3]:
latminN=30
latmaxN=40
lonminN=-55
lonmaxN=-40


In [4]:
latminM=35.1
latmaxM=44.4
lonminM=-5.7
lonmaxM=9.6


In [5]:
tdir='/work/ALT/swot/aval/wisa/tmp/inland_cleaning/swot_simulator_'+phase+'/'+simu+'-SSH-1h/'+data
files=glob.glob(tdir+'/2009/*nc')

In [6]:
ds=xr.open_mfdataset(files,concat_dim='num_lines',combine='nested')

In [7]:
ds

<xarray.Dataset>
Dimensions:                (num_lines: 20008635, num_pixels: 71)
Coordinates:
    latitude               (num_lines, num_pixels) float64 dask.array<chunksize=(9866, 71), meta=np.ndarray>
    longitude              (num_lines, num_pixels) float64 dask.array<chunksize=(9866, 71), meta=np.ndarray>
Dimensions without coordinates: num_lines, num_pixels
Data variables:
    time                   (num_lines) datetime64[ns] dask.array<chunksize=(9866,), meta=np.ndarray>
    ssh_karin              (num_lines, num_pixels) float64 dask.array<chunksize=(9866, 71), meta=np.ndarray>
    err_wet_troposphere    (num_lines, num_pixels) float64 dask.array<chunksize=(9866, 71), meta=np.ndarray>
    ssh_karin_true         (num_lines, num_pixels) float64 dask.array<chunksize=(9866, 71), meta=np.ndarray>
    err_timing             (num_lines, num_pixels) float64 dask.array<chunksize=(9866, 71), meta=np.ndarray>
    err_baseline_dilation  (num_lines, num_pixels) float64 dask.array<chunksize=(9866, 71), meta=np.ndarray>
    err_phase              (num_lines, num_pixels) float64 dask.array<chunksize=(9866, 71), meta=np.ndarray>
    err_karin              (num_lines, num_pixels) float64 dask.array<chunksize=(9866, 71), meta=np.ndarray>
    err_roll               (num_lines, num_pixels) float64 dask.array<chunksize=(9866, 71), meta=np.ndarray>
Attributes:
    Conventions:                        CF-1.7
    title:                              Level 2 Low Rate Sea Surface Height D...
    institution:                        CNES/JPL
    source:                             Simulate product
    history:                            2020-07-25 12:59:08 : Creation
    platform:                           SWOT
    references:                         Gaultier, L., C. Ubelmann, and L.-L. ...
    reference_document:                 D-56407_SWOT_Product_Description_L2_L...
    contact:                            CNES aviso@altimetry.fr, JPL podaac@p...
    cycle_number:                       7
    pass_number:                        414
    time_coverage_start:                2009-11-16T23:37:13.365167Z
    time_coverage_end:                  2009-11-17T00:28:39.792495Z
    time_coverage_duration:             P51M26S
    time_coverage_resolution:           P1S
    geospatial_lon_min:                 0.0008191316746889289
    geospatial_lon_max:                 359.9987227733048
    geospatial_lat_min:                 -78.29163662969168
    geospatial_lat_max:                 78.29196356426404
    left_first_longitude:               198.50888524328604
    left_first_latitude:                77.03291420683716
    left_last_longitude:                5.463758936048676
    left_last_latitude:                 -78.29163662969168
    right_first_longitude:              198.5156550831449
    right_first_latitude:               78.29196356426404
    right_last_longitude:               5.464618840547371
    right_last_latitude:                -77.03258645681888
    wavelength:                         0.008385803020979
    orbit_solution:                     POE
    xref_input_l1b_lr_intf_file:        N/A
    xref_input_static_karin_cal_file:   N/A
    xref_input_static_geophys_files:    N/A
    xref_input_dynamic_geophys_files:   N/A
    xref_input_l1_nalt_gdr_files:       N/A
    xref_input_int_lr_xover_cal_files:  N/A
    xref_input_l2_rad_gdr_files:        N/A
    ellipsoid_semi_major_axis:          1.0
    ellipsoid_flattening:               0.0

In [1]:
ssh=ds.ssh_karin
lat=ds.latitude
lon=ds.longitude
lon[np.where(lon>180)]=lon-360

NameError: name 'ds' is not defined

In [ ]:
fig=plt.figure(figsize=(20,15))
ax = plt.subplot(111,projection=ccrs.PlateCarree(central_longitude=0))
ax.set_extent((-100, 46, 6, 67))
ax.coastlines(resolution="10m")
gl = ax.gridlines(draw_labels=True, linestyle=':', color='black',
                      alpha=0.5)
gl.top_labels = False
gl.right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
ax.tick_params('both',labelsize=22)

ax.plot([lonminN, lonminN], [latminN, latmaxN],color='black',linewidth=3)
ax.plot([lonmaxN, lonmaxN], [latminN, latmaxN],color='black',linewidth=3)
ax.plot([lonminN, lonmaxN], [latminN, latminN],color='black',linewidth=3)
ax.plot([lonminN, lonmaxN], [latmaxN, latmaxN],color='black',linewidth=3)    
ax.plot([lonminM, lonminM], [latminM, latmaxM],color='black',linewidth=3)
ax.plot([lonmaxM, lonmaxM], [latminM, latmaxM],color='black',linewidth=3)
ax.plot([lonminM, lonmaxM], [latminM, latminM],color='black',linewidth=3)
ax.plot([lonminM, lonmaxM], [latmaxM, latmaxM],color='black',linewidth=3)    

for f in np.arange(len(files)):
    file=files[f]
    nfile=file.split('/')[-1]
    nnfile=nfile.split('.')[0]
    filename='/work/ALT/odatis/eNATL60/alberta/SWOT-sim/'+phase+'/latlon_debut_track_'+nnfile+'.npz'
    if os.path.exists(filename):
        spec = np.load('/work/ALT/odatis/eNATL60/alberta/SWOT-sim/'+phase+'/latlon_debut_track_'+nnfile+'.npz')
        lat_read = spec['lat_nonan'];
        lon_read = spec['lon_nonan'];
        plt.scatter(lon_read,lat_read, c='g', linewidth=0, s=18);
